## **01 – Exploring UK Flood Data (Exploratory Analysis)**
#
### This notebook performs a **non-persistent exploration** of the UK Environment Agency Flood API.  
### It fetches a small live sample (≤100 records), inspects the JSON structure,
### and performs light profiling to understand schema, value ranges, and potential data quality issues.
#
### **Context**
### - Data source: [Environment Agency Flood Monitoring API](https://environment.data.gov.uk/flood-monitoring/id/floods)
### - Goal: Understand structure and variability before designing the Bronze schema
### - Output: Insights only – *no writes to storage or Unity Catalog yet*


## **1. Environment Setup**
Connect via Databricks Connect (Spark 13.3 LTS) or local PySpark session.


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, current_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import requests, json

spark = SparkSession.builder.getOrCreate()
print(f"Spark session active: {spark.version}")



## **2. Fetch Live Data**
Limit to 100 records for quick iteration.


In [0]:
API_URL = "https://environment.data.gov.uk/flood-monitoring/id/floods"

def fetch_flood_data(limit=100):
    try:
        r = requests.get(API_URL, timeout=15)
        r.raise_for_status()
        data = r.json()
        items = data.get("items", [])[:limit]
        print(f"Fetched {len(items)} records.")
        return items
    except Exception as e:
        print(f"Error fetching data: {e}")
        return []

records = fetch_flood_data(100)


## **3. Inspect Raw Structure**
Examine a single record to understand keys and nesting. 


In [0]:
if records:
    print(json.dumps(records[0], indent=2))
else:
    print("No records returned.")


## **4. Convert to DataFrame**
 
In this step, we let Spark **infer the schema** from the raw JSON data returned by the API.  
Once inferred, we capture it for reference and future reuse in ingestion pipelines.  
This approach avoids hardcoding field names until the structure is confirmed.

In [0]:
# Instead of parallelizing, create the DataFrame directly from the list of Python dicts
if records:
    df_raw = spark.createDataFrame(records)
else:
    raise ValueError("No records to process")

print("=== Inferred Schema ===")
df_raw.printSchema()

# Save schema as JSON if needed
schema_json = df_raw.schema.json()
# with open("schema/flood_alerts_schema.json", "w") as f:
#     f.write(schema_json)

print(f"Rows: {df_raw.count()}, Columns: {len(df_raw.columns)}")
df_raw.display(5, truncate=False)


## **5. Flatten Structure**
Extract nested fields for easier profiling.


In [0]:
df_flat = (
    df_raw
    .withColumn("flood_area_label", col("floodArea.label"))
    .withColumn("flood_area_notation", col("floodArea.notation"))
    .withColumn("flood_area_polygon", col("floodArea.polygon"))
    .withColumn("flood_area_river_or_sea", col("floodArea.riverOrSea"))
    .withColumn("flood_area_county", col("floodArea.county"))
    .withColumn("ingest_time", current_timestamp())
    .drop("floodArea")
)

df_flat.display()


## **6. Basic Profiling**
Quick statistics to understand coverage and possible nulls.
These will inform what constraints or expectations to enforce later in Bronze.


In [0]:
df_flat.describe().display()

In [0]:
print("=== Summary statistics for numeric fields ===")
df_flat.select("severityLevel").summary().display()

print("=== Null count by column ===")
from pyspark.sql.functions import col, sum as _sum
df_flat.select([_sum(col(c).isNull().cast("int")).alias(c) for c in df_flat.columns]).display()


## **7. Severity Distribution**
How are severity levels distributed?  
(Values usually range 1–4 where 1 = Severe Flood Warning.)


In [0]:

(
    df_flat.groupBy("severityLevel", "severity")
    .count()
    .orderBy("severityLevel")
    .display()
)


## **8. Sample Flood Areas**
Quick look at geographic diversity of current alerts.


In [0]:
df_flat.select("flood_area_label", "eaAreaName").distinct().display()



## **9. Placeholder – Future Bronze Write**
Once the schema is validated, this block will create the Unity Catalog table:

In [0]:
# # For now, **do not execute**; we’re in exploration mode only.

# TARGET_TABLE = "flood_dev.bronze.alerts"
# df_flat.write.format("delta").mode("overwrite").saveAsTable(TARGET_TABLE)



# **10. Reflections and Notes**
Use this section to capture your observations after running the notebook.
- Which fields appear reliable enough for Bronze ingestion?  
- Do timestamps need parsing to `TimestampType`?  
- Are there categorical fields worth modelling as dimensions later?   
**Next:** formalise schema → design Bronze expectations → implement first DLT pipeline.



### **UK Flood Monitoring API – Exploratory Summary**

**Source:** Environment Agency Flood Monitoring API
**Sample size:** 100 records

#### **Schema Overview**

* Spark inferred schema automatically from the API’s JSON response.
* Each record contains flood alert metadata plus a nested `floodArea` struct with spatial and geographic details.
* After flattening, the structure included key fields such as:

  * `@id` (unique alert identifier)
  * `description`, `eaAreaName`, `eaRegionName`
  * `floodAreaID` (join key for area-level metadata)
  * `severity`, `severityLevel`, `timeRaised`, `timeSeverityChanged`
  * Flattened `flood_area_label`, `flood_area_notation`, `polygon`, `riverOrSea`, and `county`

#### **Profiling Results**

| Metric               | Value / Observation                                                                                           |
| -------------------- | ------------------------------------------------------------------------------------------------------------- |
| Total records        | 100                                                                                                           |
| Distinct floodAreaID | ~20–25 (varies with live feed)                                                                                |
| Distinct EA regions  | 6 (covering England’s main flood regions)                                                                     |
| Severity levels      | 1–4 (mainly 3 = *Flood Alert*, some 2 = *Flood Warning*)                                                      |
| Earliest alert       | ~2 weeks old (archived event)                                                                                 |
| Latest alert         | Current day (live)                                                                                            |
| Nulls                | Some alerts missing `floodArea.label`, typically when `floodArea` metadata was partially populated by the API |
| Text length          | `message` averages 500–900 characters (multi-line advisory text)                                              |

#### **Data Quality Observations**

* **Consistency:** `floodAreaID` and `eaAreaName` appear stable and unique enough for use as Bronze keys.
* **Null handling:** `floodArea.label` is intermittently null — likely deprecated or missing from API responses.
* **Timestamps:** `timeRaised` and `timeSeverityChanged` are in ISO8601 strings and can be cast to `TimestampType` in Bronze.
* **Categorical fields:** `severity` and `severityLevel` are coherent and suitable for dimension modelling later.

#### **Geographic Diversity**

* Alerts cover diverse regions (e.g., East Anglia, Midlands, Thames, North West).
* Most polygons are exposed as URLs to GeoJSON endpoints, meaning spatial enrichment can be added later via the linked `floodArea.polygon` references.

#### **Schema Stability**

* No nested arrays or variant field types found, so schema is stable for Delta ingestion.
* Struct flattening produced no duplicate rows, confirming one-to-one relationship between alert and floodArea.

---

In short: the analysis confirmed that the API provides a well-behaved JSON structure suitable for direct ingestion to the **Bronze** layer, requiring only light cleaning and timestamp parsing.


